In [1]:
from __future__ import division
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import os

from datetime import datetime


cwd = os.getcwd()

parent = os.path.dirname(cwd)

parent

'/Users/juju/Library/Mobile Documents/com~apple~CloudDocs/ta_project'

In [2]:
#city_name = "Miami"

In [3]:
# Import basic information data
#str_info = pd.read_excel(os.path.join(parent+r"/data/from_str_data",r"Basic Property Info {}.xlsx".format(city_name)))

In [4]:
str_info = pd.read_excel(os.path.join(parent+r"/data/STR 2022",r"master_file_SHAREID.xlsx"))

In [5]:
str_info = str_info[["SHARE ID","Market","Operation","Class","Location","Open Date"]]


In [11]:
str_info[str_info['Market']!="Boston, MA"]["SHARE ID"].nunique()

3056

In [10]:
str_info["Market"].unique()

array(['Miami, FL', 'Nashville, TN', 'Houston, TX', 'Atlanta, GA',
       'Dallas, TX', 'New York, NY', 'Boston, MA', 'Chicago, IL',
       'Washington, DC', 'Phoenix, AZ'], dtype=object)

In [21]:
# Import revenue Data
str_revenue = pd.read_excel(os.path.join(parent+r"/data/STR 2022",r"Master_file_MonthlyPerformanceData_200001_202212.xlsx".format(city_name)))

In [22]:
str_revenue

,SHARE ID,Year Month,Occ,ADR,RevPAR
0,51622,202205,65.3763,179.237458,117.178897
1,51622,202206,81.6825,169.175884,138.187159
2,51622,202207,79.1705,165.909926,131.351730
3,51622,202208,92.2273,174.633604,161.059932
4,51622,202209,88.6349,169.458188,150.199130
...,...,...,...,...,...
409157,28650555,202208,70.8714,180.239130,127.738084
409158,28650555,202209,61.5422,170.759095,105.089055
409159,28650555,202210,60.8101,197.896707,120.341375
409160,28650555,202211,58.2222,230.335115,134.106222


In [23]:
str_data = str_revenue.merge(str_info, on="SHARE ID")

In [24]:
str_data.head()

,SHARE ID,Year Month,Occ,ADR,RevPAR,Market,Operation,Class,Location,Open Date
0,51622,202205,65.3763,179.237458,117.178897,"Nashville, TN",Chain,Upper Midscale,Urban,2022.0
1,51622,202206,81.6825,169.175884,138.187159,"Nashville, TN",Chain,Upper Midscale,Urban,2022.0
2,51622,202207,79.1705,165.909926,131.351730,"Nashville, TN",Chain,Upper Midscale,Urban,2022.0
3,51622,202208,92.2273,174.633604,161.059932,"Nashville, TN",Chain,Upper Midscale,Urban,2022.0
4,51622,202209,88.6349,169.458188,150.199130,"Nashville, TN",Chain,Upper Midscale,Urban,2022.0


In [25]:
# convert string to date month
str_data['year_month'] = str_data["Year Month"].astype(str).str[:4]+'-'+\
                         str_data["Year Month"].astype(str).str[-2:]

In [26]:
str_data['year_month'] = pd.to_datetime(str_data['year_month']).dt.strftime('%Y-%m')

In [27]:
str_data["year_month"] = pd.to_datetime(str_data["year_month"]).dt.to_period('M')

In [28]:
str_data.columns

Index(['SHARE ID', 'Year Month', 'Occ', 'ADR', 'RevPAR', 'Market', 'Operation',
       'Class', 'Location', 'Open Date', 'year_month'],
      dtype='object')

In [29]:
# Number of months of data from each hotel

str_data.groupby("SHARE ID")["Year Month"].nunique().describe()

count    2491.000000
mean      164.256122
std        98.412803
min         2.000000
25%        66.000000
50%       171.000000
75%       269.000000
max       276.000000
Name: Year Month, dtype: float64

In [30]:
#### Some months don't have any revenue data
#### Generate a dataframe without month gaps
#### Leave performance columns as NA

#performance_cols = [ 'Supply', 'Demand', 'Revenue', 'Occ', 'ADR','RevPAR' ]

performance_cols = ['Occ', 'ADR','RevPAR' ]
l=[]

hotels=str_data.groupby("SHARE ID")

for name, h in hotels:
    h_resample = h.set_index('year_month').resample('M').asfreq().ffill().reset_index(level=0, drop=False)
    h_resample["month_with_revenue"] = h_resample["year_month"].isin(h["year_month"])
    
    for col in performance_cols:
    
        h_resample[col]= np.where(h_resample["month_with_revenue"]==False, 
                      np.nan,h_resample[col])
    l.append(h_resample)
    
str_monthly = pd.concat(l,axis=0)

In [31]:
str_monthly = str_monthly.reset_index().drop(["index"],axis=1)

In [47]:
missing_revenue_months = str_monthly.groupby("SHARE ID").apply(lambda x: sum(x["Occ"].isna()))

In [48]:
missing_revenue_months = missing_revenue_months.reset_index().rename(columns={0:"number of missing months"})

In [49]:
missing_revenue_hotels = missing_revenue_months[missing_revenue_months["number of missing months"]!=0]

In [50]:
missing_revenue_hotels

,SHARE ID,number of missing months
2,75909.0,2
4,80055.0,85
5,80076.0,1
8,80286.0,1
11,80489.0,12
...,...,...
2466,10117030.0,2
2467,10123160.0,16
2470,10137690.0,2
2473,10155434.0,125


In [20]:
#output_dir = parent+r"/data/missing_revenue_hotels"
#missing_revenue_hotels\
   # .to_excel(os.path.join(output_dir,r"{}_hotels_missing_revenues.xlsx".format(city_name)), index = False, encoding='utf-8')

In [51]:
# Drop hotels with more than 3 months of missing revenue

drop_hotels = missing_revenue_hotels

In [52]:
print('Total number of hotels have missing revenues from at least 3 months (i.e. gaps in monthly revenue) from {} ({}%)--{}'.format(*[\
                                                    len(missing_revenue_hotels),\
                                                    round(len(missing_revenue_hotels)/str_data["SHARE ID"].nunique(),3)*100,\
                                                    city_name]))
print('Dropped {} ({}%) Hotels From {}.'.format(*[drop_hotels["SHARE ID"].nunique(),\
                                                 100*round(drop_hotels["SHARE ID"].nunique()/str_data["SHARE ID"].nunique(),3),\
                                                 city_name]))

Total number of hotels have missing revenues from at least 3 months (i.e. gaps in monthly revenue) from 1075 (43.2%)--Miami
Dropped 1075 (43.2%) Hotels From Miami.


In [53]:
str_monthly_cleaned = str_monthly[~str_monthly["SHARE ID"].isin(drop_hotels["SHARE ID"])]

In [54]:
str_monthly_cleaned["SHARE ID"].nunique()

1416

In [55]:
str_monthly_cleaned = str_monthly_cleaned.reset_index().drop(["index"],axis=1)

In [60]:
str_monthly_cleaned["Market"].unique()

array(['Nashville, TN', 'Miami, FL', 'Houston, TX', 'Dallas, TX',
       'New York, NY', 'Boston, MA', 'Chicago, IL', 'Washington, DC',
       'Phoenix, AZ', 'Atlanta, GA'], dtype=object)

In [43]:
#str_monthly_cleaned = str_monthly_cleaned.drop(["year_month","Country","Market","month_with_revenue"],axis=1)


In [57]:
# import the KNNimputer class
#from sklearn.impute import KNNImputer

#imputer = KNNImputer(n_neighbors=5)
#str_monthly_filled = imputer.fit_transform(str_monthly_cleaned)

  

In [ ]:
#str_monthly_filled = pd.DataFrame(str_monthly_filled,columns=list(str_monthly_cleaned.columns))

In [29]:
#str_monthly_cleaned = str_monthly[~str_monthly["SHARE ID"].isin(drop_hotels["SHARE ID"])]
#str_monthly_cleaned = str_monthly_cleaned.reset_index().drop(["index"],axis=1)
#str_monthly_filled["year_month"] = str_monthly_cleaned["year_month"]

In [61]:
str_monthly_cleaned

,year_month,SHARE ID,Year Month,Occ,ADR,RevPAR,Market,Operation,Class,Location,Open Date,month_with_revenue
0,2022-05,51622.0,202205.0,65.3763,179.237458,117.178897,"Nashville, TN",Chain,Upper Midscale,Urban,2022.0,True
1,2022-06,51622.0,202206.0,81.6825,169.175884,138.187159,"Nashville, TN",Chain,Upper Midscale,Urban,2022.0,True
2,2022-07,51622.0,202207.0,79.1705,165.909926,131.351730,"Nashville, TN",Chain,Upper Midscale,Urban,2022.0,True
3,2022-08,51622.0,202208.0,92.2273,174.633604,161.059932,"Nashville, TN",Chain,Upper Midscale,Urban,2022.0,True
4,2022-09,51622.0,202209.0,88.6349,169.458188,150.199130,"Nashville, TN",Chain,Upper Midscale,Urban,2022.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...
214985,2022-08,28650555.0,202208.0,70.8714,180.239130,127.738084,"Miami, FL",Independent,Upper Upscale,Urban,2021.0,True
214986,2022-09,28650555.0,202209.0,61.5422,170.759095,105.089055,"Miami, FL",Independent,Upper Upscale,Urban,2021.0,True
214987,2022-10,28650555.0,202210.0,60.8101,197.896707,120.341375,"Miami, FL",Independent,Upper Upscale,Urban,2021.0,True
214988,2022-11,28650555.0,202211.0,58.2222,230.335115,134.106222,"Miami, FL",Independent,Upper Upscale,Urban,2021.0,True


In [31]:
#str_monthly_filled = str_monthly_filled[['SHARE ID', 'year_month','Supply', 'Demand', 'Revenue', 'Occ', 'ADR', 'RevPAR']]

In [32]:
#str_monthly_merge = str_monthly[['SHARE ID','Country', 'Market', 'Chain', 'Owner', 'Mgmt',
      # 'ParentCompany', 'Operation', 'Scale', 'Class', 'Location', 'OpenDate',
      # 'SizeCode','year_month']]

In [33]:
#str_df = str_monthly_filled.merge(str_monthly_merge, on=["SHARE ID","year_month"], how="left")

In [36]:
#str_df["SHARE ID"].nunique()

314

In [35]:
#str_df.to_csv(os.path.join(parent+r"/data/cleaned_str_data",r"{}_STR.csv".format(city_name)), index= False)

In [58]:
str_monthly_cleaned.to_csv(os.path.join(parent+r"/data/cleaned_str_data",r"{}_STR_2022.csv"), index= False)